#### __11. 심층 신경망 훈련하기__

__11.1 그레이디언트 소실(vanishing gradient)과 폭주(exploding gradient)__

역전파 알고리즘은 출력층에서 입력층으로 오차 그레이디언트를 전파하면서 진행된다. <br>
알고리즘이 신경망의 모든 파라미터에 대한 오차 함수의 그레이디언트를 계산하면, <br>
경사 하강법 단계에서 이 그레이디언트를 사용하여 각 파라미터를 수정한다. <br>
그런데 알고리즘이 하위층으로 진행될수록 그레이디언트가 점점 작아지는 경우가 많다. <br>
경사 하강법이 하위층의 연결 가중치를 변경되지 않은 채로 둔다면 훈련이 수렴하지 않을 것이다. <br>
이를 그레이디언트 소실이라고 한다. 반대로 그레이디언트가 점점 커져서 여러 층이 비정상적으로 <br>
큰 가중치로 갱신되면 알고리즘은 발산하고, 이를 그레이디언트 폭주라고 한다.

In [2]:
import os
import sys
 
import sklearn
import tensorflow as tf

import numpy as np
np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
keras = tf.keras

In [3]:
# 텐서와 연산
tf.constant([[1., 2., 3.], [4., 5., 6.]])  # 행렬

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
tf.constant(42)  # 스칼라

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [6]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [9]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [10]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [11]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [14]:
t @ tf.transpose(t) # @는 행렬 곱셈

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [15]:
# keras.backend
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

In [17]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [18]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [19]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [20]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [22]:
# 타입 변환
# 텐서플로는 어떤 타입 변환도 자동으로 수행하지 않고, 호환되지 않는 타입의 텐서로 연산을 실행하면 예외 발생
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [23]:
# 넘파이는 기본으로 64비트 정밀도를 사용하지만, 텐서폴르논 32비트 정밀도를 사용
# 32비트 실수와 64비트 실수도 더할 수 없다
try:
    tf.constant(2.0) + tf.constant(40., dtype=tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [24]:
# 타입 변환이 필요할 때는 tf.cast() 함수를 사용할 수 있다
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [31]:
# 변수
# tf.Tensor는 변경이 불가능한 객체이다. 그런데 역전파로 변경되어야 하는 가중치를 구현하기 위해서는 변수가 필요하다.
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [32]:
# assign() 메서드를 사용하여 변숫값을 바꿀 수 있다
# 직접 수정은 안된다
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [27]:
v[0, 1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [28]:
v[:, 2].assign([0., 1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [29]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

__12.3 사용자 정의 손실 함수__

In [34]:
# 캘리포니아 주택 데이터셋 로드
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [35]:
# 후버Huber 손실 구현
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [36]:
input_shape = X_train.shape[1:]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

In [43]:
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 7ms/step - loss: 0.1977 - val_loss: 0.1927
Epoch 2/2
363/363 [==============================] - 3s 7ms/step - loss: 0.1927 - val_loss: 0.1867


In [44]:
# 사용자 정의 요소를 가진 모델을 저장하고 로드하기
model.save("my_model_with_a_custom_loss.h5")

In [45]:
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [47]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 8ms/step - loss: 0.1893 - val_loss: 0.1737
Epoch 2/2
363/363 [==============================] - 3s 7ms/step - loss: 0.1866 - val_loss: 0.1778


In [49]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

In [50]:
model.compile(loss=create_huber(2.0), optimizer="nadam", metrics=["mae"])

In [51]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 7ms/step - loss: 0.2082 - mae: 0.4687 - val_loss: 0.2024 - val_mae: 0.4545
Epoch 2/2
363/363 [==============================] - 3s 7ms/step - loss: 0.2053 - mae: 0.4668 - val_loss: 0.1900 - val_mae: 0.4489


In [52]:
model.save("my_model_with_a_custom_loss_threshold_2.h5")

In [53]:
# 모델을 저장할 때 threshold 값은 저장되지 않으므로, 로드 후 값을 지정해야 한다
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

In [54]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 8ms/step - loss: 0.2031 - mae: 0.4640 - val_loss: 0.1935 - val_mae: 0.4515
Epoch 2/2
363/363 [==============================] - 3s 8ms/step - loss: 0.2013 - mae: 0.4615 - val_loss: 0.2156 - val_mae: 0.4622


In [55]:
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [56]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal",
                       input_shape=input_shape),
    keras.layers.Dense(1),
])

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
363/363 [==============================] - 3s 7ms/step - loss: 0.7876 - mae: 0.9624 - val_loss: 0.3110 - val_mae: 0.5432
Epoch 2/2
363/363 [==============================] - 3s 8ms/step - loss: 0.2532 - mae: 0.5228 - val_loss: 0.2657 - val_mae: 0.5130


In [57]:
# 모델 저장 시 임곗값도 함께 저장
model.save("my_model_with_a_custom_loss_class.h5")

In [ ]:
model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
                                custom_objects={"HuberLoss": HuberLoss})

model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))